# Update a Model Notebook

Updating is the process of re-training an ESM from a dataset. While induction creates the structure of a model, including the modules, partitions and rules (see Understand ESMs) and trains it, re-training does not change the model structure but re-trains the neural models in each rule.

In this notebook we will induce an ESM and then update it.

# Check Environment Variables
Before installing Hybrid Intelligence in the notebook you need to set these Environment Variables externally as described in the User Guide https://docs.umnai.com/set-up-your-environment. 
This section checks that the environment variables have been set correctly and throws an error if not.

In [1]:
import os

umnai_env_vars = {
    'UMNAI_CLIENT_ID',
    'UMNAI_CLIENT_SECRET',
    'PIP_EXTRA_INDEX_URL',
}

if os.environ.get('DATABRICKS_RUNTIME_VERSION') is not None:
    umnai_env_vars.update({
        'DATABRICKS_HOST',
        'DATABRICKS_TOKEN',
    })

if not umnai_env_vars.issubset(os.environ.keys()):
    raise ValueError(
        'UMNAI environment variables not set correctly. They need to be set before using the Umnai library.'
    )

# Install Hybrid Intelligence
Next we install the UMNAI Platform.

In [2]:
%pip install umnai-platform --quiet


[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


# Set Workspace Paths According to Your Environment
Now we will set the workspace path and the experiment path automatically. They will be set to a local path if you are using a local machine environment or to a Databricks path if you are using a Databricks environment.

## Install Databricks SDK

This checks if you are running on Databricks and installs their SDK if you are.

In [3]:
import os
if os.environ.get('DATABRICKS_RUNTIME_VERSION') is not None:
    %pip install databricks-sdk --quiet

If you are on Databricks, you can select whether you would like the workspace to be created in the shared area (available to all users in your account) or in your personal user account area. You can ignore this if you are running on a local environment.

In [4]:
# Set to 1 if you want to use shared or 0 to use personal user account area.
USE_SHARED_WORKSPACE = 1 

## Set Paths
Next the workspace and experiment paths are set automatically.

In [5]:
import os

EXP_NAME = 'modelupdate_adult_income'
if os.environ.get('DATABRICKS_RUNTIME_VERSION') is not None:
    from databricks.sdk import WorkspaceClient
    w = WorkspaceClient()

    # # For a Databricks Environment
    WS_PATH = '/dbfs/FileStore/workspaces/'+EXP_NAME
    if USE_SHARED_WORKSPACE:
        EXP_PREFIX = f'/Shared/experiments/'
    else:
        USERNAME = dbutils.notebook.entry_point.getDbutils().notebook().getContext().userName().get()
        EXP_PREFIX = f'/Users/{USERNAME}/experiments/'
    w.workspace.mkdirs(EXP_PREFIX)
    EXP_PATH = EXP_PREFIX + EXP_NAME
else:
    # For a Local Machine Environment
    WS_PATH = 'resources/workspaces/'+EXP_NAME
    EXP_PATH = EXP_NAME

# Import and Prepare Dataset
Import the dataset to a Pandas DataFrame and the clean data in preparation for onboarding into Hybrid Intelligence.

In [6]:
import pandas as pd
import numpy as np

# Import Adult Income Dataset to pandas dataframe: 
# This dataset can be downloaded from https://archive.ics.uci.edu/dataset/2/adult 
column_names = ["Age", "WorkClass", "fnlwgt", "Education", "EducationNum", "MaritalStatus", "Occupation", "Relationship", "Race", "Gender", "CapitalGain", "CapitalLoss", "HoursPerWeek", "NativeCountry", "Income"]
dataset_df = pd.read_csv('https://raw.githubusercontent.com/umnaibase/umnai-examples/main/data/adult.data', names = column_names)

# Data Preparation:
dataset_df = dataset_df.apply(lambda x: x.str.strip() if x.dtype == 'object' else x)    # Remove whitespaces
dataset_df["Income"] = np.where((dataset_df["Income"] == '<=50K'), 0, 1)                # Replace Target values with [0,1]
dataset_df.head(5)


,Age,WorkClass,fnlwgt,Education,EducationNum,MaritalStatus,Occupation,Relationship,Race,Gender,CapitalGain,CapitalLoss,HoursPerWeek,NativeCountry,Income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


# Split the Data

In this example we will use the first 25000 data rows to induce the model and then we will use the full dataset to update the model.

In [7]:
induction_dataset_df = dataset_df.iloc[:25000].reset_index()
induction_dataset_df.head(5)

,index,Age,WorkClass,fnlwgt,Education,EducationNum,MaritalStatus,Occupation,Relationship,Race,Gender,CapitalGain,CapitalLoss,HoursPerWeek,NativeCountry,Income
0,0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


# Create or Open a Hybrid Intelligence Workspace

In [8]:
from umnai.workspaces.context import Workspace

# Open a workspace
ws = Workspace.open(
    path = WS_PATH,
    experiment= EXP_PATH
)

ws # Prints workspace details to confirm created/opened

# Onboard The Induction Dataset

Onboard the Pandas DataFrame into a Hybrid Intelligence dataset.

### Activate Masking
Activating masking (handling of unseen categories) on categorical features can be beneficial when a model is going to be updated with new data. This is because the new data might contain categories that were not present in the original training data. By masking unseen categories, the model will be capable of handling any new categories making it more flexible and robust.

In [9]:
from umnai.data.datasets import Dataset
from umnai.data.enums import PredictionType

dataset = Dataset.from_pandas(
    induction_dataset_df,
    prediction_type=PredictionType.CLASSIFICATION,
    features=list(dataset_df.drop(['Income'], axis=1).columns),    # All columns except 'Income' are features
    targets=['Income'],
    handle_unseen_categories=True      # Activate handle unseen categories for all categorical features
)

dataset # Prints dataset details to confirm created/opened

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/01/17 11:49:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting Spark log level to "WARN".


[DatasetInitialization] - MLFLOW Run ID: f7335809dd164e6882999f4d9fe95a5a:   0%|          | 0/42 [00:00<?, ?it…

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


INFO:tensorflow:Assets written to: /opt/atlassian/pipelines/agent/build/demo-notebooks/resources/workspaces/modelupdate_adult_income/preprocessing/dataset_name=Dataset_d076bb28/assets


Dataset(id=eb0e9542-bf2c-48a4-bfb4-1d1fc0e7f10c; name=Dataset_d076bb28; is_named=False; workspace_id=None)

# Induce a Hybrid Intelligence Model

First we induce a model using the first part of the dataset.

Pre-induced models are available in the notebook workspace on Github and may be downloaded and saved locally. Using Pre-induced models will speed up the execution of the notebook.

If `LOAD_PREINDUCED_MODEL` is set to `1` (default), the notebook will look for and load the pre-induced model with `ESM_ID`. Otherwise, if set to `0` or the pre-induced model is not found, a new model will be induced and saved to the workspace.

In [10]:
# Set this variable to '1' to load a pre-induced model, otherwise set to '0' to re-induce a new model from the dataset
LOAD_PREINDUCED_MODEL = 1

# Model ID
ESM_ID='Dataset_5024bf0c_3d01d372820742dea7a9d36ff2ecf6f0'

#### Load or Induce the Model

In [11]:
from umnai.esm.model import ESM
from umnai.induction.inducer import ModelInducer

# Check if a saved model with the ESM_ID exists. If it exists load it, otherwise induce a new model, save it and print the model and run IDs
if (LOAD_PREINDUCED_MODEL == 1):
    try:    
        esm = ESM.from_workspace(id = ESM_ID)
        print('Pre-induced ESM loaded from workspace: ' + esm.id)
    except OSError:
        print("No model found in workspace.")
        LOAD_PREINDUCED_MODEL = 0

if (LOAD_PREINDUCED_MODEL == 0):
    print("Inducing a new model - this may take some time.")
    # Induce a simple model quickly using fast execution parameters
    model_inducer = ModelInducer(
        max_interactions=3,
        max_interaction_degree=2,
        max_polynomial_degree=2,
        trials=2,
        estimators=2,
        batch_size=512,
        iterations=2,
    )

    # # Induce a more realistic model using default Induction parameters:
    # model_inducer = ModelInducer()

    # Create an ESM using Induction
    esm = model_inducer.induce(dataset)

    # Save the ESM to your workspace
    esm.save_to_workspace()

    # Note ESM ID and MLFLow Run ID
    print("ESM ID: ", esm.id)
    print("MLflow Run ID: ", esm.producer_run_id)


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


...

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Pre-induced ESM loaded from workspace: Dataset_5024bf0c_3d01d372820742dea7a9d36ff2ecf6f0


# Update the Model

Next we will update the model using the full dataset.

# Onboard the Updated Dataset

Onboard the full dataset.

In [12]:
from umnai.data.datasets import Dataset
from umnai.data.enums import PredictionType

updated_dataset = Dataset.from_pandas(
    dataset_df,
    prediction_type=PredictionType.CLASSIFICATION,
    features=list(dataset_df.drop(['Income'], axis=1).columns),    # All columns except 'Income' are features
    targets=['Income'],
    categorical_features=list(dataset_df.select_dtypes(object).columns),    # All 'object' columns are categorical here
    handle_unseen_categories=True      # Activate handle unseen categories for all categorical features
)

updated_dataset # Prints dataset details to confirm created/opened

[Masking] - MLFLOW Run ID: 5df4d30b72a34c55a2387bd3e5af0c73:   0%|          | 0/42 [00:00<?, ?it/s]

INFO:tensorflow:Assets written to: /opt/atlassian/pipelines/agent/build/demo-notebooks/resources/workspaces/modelupdate_adult_income/preprocessing/dataset_name=Dataset_c0ae8909/assets


Dataset(id=baf9ce10-6a57-4820-af95-548d7c0b0633; name=Dataset_c0ae8909; is_named=False; workspace_id=None)

# Instantiate a ModelUpdater and Update the Model

# Update the Model

Next we update the model using the updated dataset.

A pre-updated model is available in the notebook workspace on Github and may be downloaded and saved locally. Using the pre-updated model will speed up the execution of the notebook.

If `LOAD_PREUPDATED_MODEL` is set to `1` (default), the notebook will look for and load the pre-updated model with `UPDATED_ESM_ID`. Otherwise, if set to `0` or the pre-updated model is not found, the current model will be updated and saved to the workspace.

In [13]:
# Set this variable to '1' to load a pre-induced model, otherwise set to '0' to re-induce a new model from the dataset
LOAD_PREUPDATED_MODEL = 1

# Model ID
UPDATED_ESM_ID='Dataset_5024bf0c_c238ae32fe274aefa1919c7e26516445'

#### Load or Induce the Model

In [14]:
from umnai.induction.updater import ModelUpdater

# Check if a saved model with the UPDATED_ESM_ID exists. If it exists load it, otherwise induce a new model, save it and print the model and run IDs
if (LOAD_PREUPDATED_MODEL == 1):
    try:    
        updated_esm = ESM.from_workspace(id = UPDATED_ESM_ID)
        print('Pre-updated ESM loaded from workspace: ' + esm.id)
    except OSError:
        print("No model found in workspace.")
        LOAD_PREUPDATED_MODEL = 0

if (LOAD_PREUPDATED_MODEL == 0):
    print("Updating existing model - this may take some time.")

    # Instantiate a Model Updater
    model_updater = ModelUpdater(esm=esm)

    # Update the Model
    updated_esm = model_updater.update(updated_dataset)

    # Save the ESM to your workspace
    updated_esm.save_to_workspace()

    # Note ESM ID and MLFLow Run ID
    print("UPDATED ESM ID: ", updated_esm.id)
    print("MLflow Run ID: ", updated_esm.producer_run_id)


Pre-updated ESM loaded from workspace: Dataset_5024bf0c_3d01d372820742dea7a9d36ff2ecf6f0


# Compare the Models

You can compare two models using the Model Comparison View.

## ModelComparisonView

The Model Comparison View is useful for comparing two models after a model update. It provides useful insights by highlighting the commonalities and differences between two ESM models.

The Model Comparison View is based on the Model Summary View and will output a full comparison of the Model Summary View for both models in the dat attribute. You can filter the full data using three different methods:
1. `summary()`: this method gives a granular raw comparison of the full data
2. `compare_modules()`: this method outputs a comparison of the modules in the two models
3. `compare_partitions()`: this method outputs a comparison of the partitions in the two models

Since the Model Updater will only retrain the neural models within each partition and does not change the structure of the model, we will examine the `compare_modules()` output to confirm that there haven't been any changes there and the `compare_partitions()` to confirm that there have been changes within the partitions.

#### Module Comparison
The columns `delta.n_partitions` and	`delta.n_params` of the output show any changes in the number of partitions, and the number of parameters. These columns should all be zero, indicating that there haven't been any changes to module structures.

In [15]:
from umnai.views.model_comparison import ModelComparisonView

# Generate the view
model_comparison_view = ModelComparisonView(baseline_model=esm, comparison_model=updated_esm)

# Compare the modules
model_comparison_view.compare_modules()

,module_operation.module_stable_label,module_operation.module_features,module_operation.edit_operation,baseline_model.module_id,baseline_model.module_index,baseline_model.n_partitions,baseline_model.n_params,baseline_model.non_zero_params,comparison_model.module_id,comparison_model.module_index,comparison_model.n_partitions,comparison_model.n_params,comparison_model.non_zero_params,delta.n_partitions,delta.n_params,delta.non_zero_params
0,module_Age,[Age],ITEM_EDITED,bcb66ab4a2dc45a060b29b902d7b98a98a1538fd931ed7...,0,3,18,18,bcb66ab4a2dc45a060b29b902d7b98a98a1538fd931ed7...,0,3,18,18,0,0,0
1,module_fnlwgt,[fnlwgt],ITEM_EDITED,a045a43c1a78a7f59141c8c97582f4b872eb9b22d19852...,1,2,12,12,a045a43c1a78a7f59141c8c97582f4b872eb9b22d19852...,1,2,12,12,0,0,0
2,module_EducationNum,[EducationNum],ITEM_EDITED,17baab363ef19d958473f48246b0d4140b9ba3927051ca...,2,2,12,12,17baab363ef19d958473f48246b0d4140b9ba3927051ca...,2,2,12,12,0,0,0
3,module_CapitalGain,[CapitalGain],ITEM_EDITED,affcca7dafe665e95412012d227438a336d3911fe0f94b...,3,1,6,6,affcca7dafe665e95412012d227438a336d3911fe0f94b...,3,1,6,6,0,0,0
4,module_CapitalLoss,[CapitalLoss],ITEM_EDITED,1f377b0f2cbccc2963b123d4d901c005a574045448c719...,4,1,6,6,1f377b0f2cbccc2963b123d4d901c005a574045448c719...,4,1,6,6,0,0,0
5,module_HoursPerWeek,[HoursPerWeek],ITEM_EDITED,c280a2723feb2639d00dd5aac8cfabb4e897deaf74c0a0...,5,1,6,6,c280a2723feb2639d00dd5aac8cfabb4e897deaf74c0a0...,5,1,6,6,0,0,0
6,module_Education,[Education],ITEM_EDITED,7c0b83c330b649b79f7f4a31857a58df4706029ab3c3c7...,6,1,36,36,7c0b83c330b649b79f7f4a31857a58df4706029ab3c3c7...,6,1,36,36,0,0,0
7,module_Gender,[Gender],ITEM_EDITED,aaedfe284049196e9d71bf8cb85a3237a99ba93e0e7a04...,7,2,16,10,aaedfe284049196e9d71bf8cb85a3237a99ba93e0e7a04...,7,2,16,10,0,0,0
8,module_MaritalStatus,[MaritalStatus],ITEM_EDITED,f7fd8b3d9cd539938f288cae9dedb42758d1e6d7ebce1b...,8,2,36,20,f7fd8b3d9cd539938f288cae9dedb42758d1e6d7ebce1b...,8,2,36,20,0,0,0
9,module_NativeCountry,[NativeCountry],ITEM_EDITED,a157455f27e05f2250d0105c4d8b3aa63cdc207ba4a374...,9,1,88,88,a157455f27e05f2250d0105c4d8b3aa63cdc207ba4a374...,9,1,88,88,0,0,0


#### Partition Comparison

The columns `delta.intercept` and	`delta.coefficients` of the output show any changes in the intercepts and coefficients of the neural models within each partition. After a model update, you should expect to see non-zero values for most partitions (assuming you are using an updated dataset).

In [16]:
model_comparison_view.compare_partitions()

,partition_operation.rule_canonical_label,partition_operation.rule_friendly_condition,partition_operation.edit_operation,baseline_partition.module_id,baseline_partition.module_index,baseline_partition.rule_id,baseline_partition.intercept,baseline_partition.coefficients,baseline_partition.transformed_features,comparison_partition.module_id,comparison_partition.module_index,comparison_partition.rule_id,comparison_partition.intercept,comparison_partition.coefficients,comparison_partition.transformed_features,delta.intercept,delta.coefficients,delta.transformed_features
0,rule_if_Age_le_31.5__rule_then_Income_Age_2_pl...,Age ≤ 31.5,ITEM_EDITED,Dataset_5024bf0c_3d01d372820742dea7a9d36ff2ecf...,0,18d837ad4e324988a26c38b1e42499cd,[-0.8186426758766174],"[2.701948324101977e-05, 1.0472211897649686e-07]","[Age, Age**2]",Dataset_5024bf0c_c238ae32fe274aefa1919c7e26516...,0,58ac22d2c8ff406db7b5a2e207003179,[-0.8651977777481079],"[2.8052518246113323e-05, 1.0232730573989102e-07]","[Age, Age**2]",[-0.04655510187149048],"[1.0330350050935522e-06, -2.3948132366058417e-09]","[0, 0]"
1,rule_if_Age_le_42.5_Age_gt_31.5__rule_then_Inc...,(Age > 31.5) and (Age ≤ 42.5),ITEM_EDITED,Dataset_5024bf0c_3d01d372820742dea7a9d36ff2ecf...,0,8881631e83014763b3e06591e13e7f24,[0.05778349190950394],"[1.2338575743342517e-06, 9.436269543527942e-09]","[Age, Age**2]",Dataset_5024bf0c_c238ae32fe274aefa1919c7e26516...,0,0f1f8dc6c1e34100905c9b5fcc4fbd66,[0.043807849287986755],"[2.0298884919611737e-06, 1.3354945593846423e-08]","[Age, Age**2]",[-0.013975642621517181],"[7.96030917626922e-07, 3.9186760503184814e-09]","[0, 0]"
2,rule_if_Age_gt_42.5__rule_then_Income_Age_2_pl...,Age > 42.5,ITEM_EDITED,Dataset_5024bf0c_3d01d372820742dea7a9d36ff2ecf...,0,193cb48efec54d5d8866429178039178,[0.2542039155960083],"[-4.840932524530217e-05, -5.077945388620719e-07]","[Age, Age**2]",Dataset_5024bf0c_c238ae32fe274aefa1919c7e26516...,0,c38d3b0e2ece4c179b99266b38c3aab2,[0.2539092004299164],"[-4.8946469178190455e-05, -5.060589955974137e-07]","[Age, Age**2]",[-0.00029471516609191895],"[-5.371439328882843e-07, 1.7355432646581903e-09]","[0, 0]"
3,rule_if_fnlwgt_le_203768.5__rule_then_Income_f...,fnlwgt ≤ 203768.5,ITEM_EDITED,Dataset_5024bf0c_3d01d372820742dea7a9d36ff2ecf...,1,c36adfbd2bfb4856a85032ce741fa0e3,[-0.0544566735625267],"[6.024854226627241e-12, 1.2149144936520527e-18]","[fnlwgt, fnlwgt**2]",Dataset_5024bf0c_c238ae32fe274aefa1919c7e26516...,1,551e180b1e15431fa3b2ffe56e73cf82,[-0.04496002942323685],"[6.336534595641208e-12, 4.843424761882011e-19]","[fnlwgt, fnlwgt**2]",[0.009496644139289856],"[3.116803690139669e-13, -7.305720174638516e-19]","[0, 0]"
4,rule_if_fnlwgt_gt_203768.5__rule_then_Income_f...,fnlwgt > 203768.5,ITEM_EDITED,Dataset_5024bf0c_3d01d372820742dea7a9d36ff2ecf...,1,fe06583fac9242b8b2d4e2fa3b5fb823,[0.0004945510299876332],"[1.5718807548983915e-12, 8.530659890519558e-19]","[fnlwgt, fnlwgt**2]",Dataset_5024bf0c_c238ae32fe274aefa1919c7e26516...,1,21be36eb6ff64ee5bfd57b54ec200b0d,[0.017260398715734482],"[5.4099186068412575e-12, 4.164841562905e-18]","[fnlwgt, fnlwgt**2]",[0.01676584768574685],"[3.838037851942866e-12, 3.311775573853044e-18]","[0, 0]"
5,rule_if_EducationNum_le_9.5__rule_then_Income_...,EducationNum ≤ 9.5,ITEM_EDITED,Dataset_5024bf0c_3d01d372820742dea7a9d36ff2ecf...,2,bd93ac88bea84507a944bf2f80200a22,[-0.762926459312439],"[0.0006083459011279047, 2.5060078769456595e-05]","[EducationNum, EducationNum**2]",Dataset_5024bf0c_c238ae32fe274aefa1919c7e26516...,2,19b3ba2f981e432e8ac6796b6360d2fc,[-0.7641474008560181],"[0.0006222190568223596, 2.5713094146340154e-05]","[EducationNum, EducationNum**2]",[-0.0012209415435791016],"[1.3873155694454908e-05, 6.530153768835589e-07]","[0, 0]"
6,rule_if_EducationNum_gt_9.5__rule_then_Income_...,EducationNum > 9.5,ITEM_EDITED,Dataset_5024bf0c_3d01d372820742dea7a9d36ff2ecf...,2,b492750563a149758f1ce7fc4d26b6b7,[0.4247683882713318],"[6.695344927720726e-05, 6.487137397925835e-06]","[EducationNum, EducationNum**2]",Dataset_5024bf0c_c238ae32fe